In [ ]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
import matplotlib.pyplot as plt
import yfinance as yf
tf.config.list_physical_devices('GPU')

In [ ]:
price_hist = yf.Ticker('NVDA').history(interval='1d', start='2020-01-01', end='2024-01-01')
price_hist.tail()
price_series = price_hist['Close']

In [ ]:
def gen_price_series(size=10000, k=3, a=0.9, instances=1):
    p_series = np.ndarray((instances, size,), dtype=np.float32)
    b_series = np.ndarray((instances, size,), dtype=np.float32)
    p_series[:, 0] = 0
    b_series[:, 0] = 0

    for i in range(1, size):
        p_series[:, i] =  p_series[:, i-1] + b_series[:, i-1] + k * np.random.normal(size=instances)
        b_series[:, i] = a * b_series[:, i-1] + np.random.normal(size=instances)

    # shape: (instances)
    R = np.max(p_series, axis=1) - np.min(p_series, axis=1)
    z_series = np.exp(p_series / np.repeat(R[:, np.newaxis], size, axis=1))

    return z_series, p_series

'''To convert real prices into model-appropriate price series:
    1. Set the initial price as 0.
    2. Scale the price to [-1, 1], with 0 as the mean.
    3. Apply exp.'''

# this function takes MULTIPLE INPUTS!!!
def normalize_price_series(price_series):
    initial_prices = price_series[:, 0]
    range = np.max(price_series, axis=1) - np.min(price_series, axis=1)
    scaled_prices = (price_series - np.repeat(initial_prices[:, np.newaxis], price_series.shape[1], axis=1)) / np.repeat(range[:, np.newaxis], price_series.shape[1], axis=1)
    
    return np.exp(scaled_prices)

a = normalize_price_series(price_series.to_numpy()[np.newaxis, :])
plt.plot(price_series.to_numpy())
plt.show()
plt.plot(a[0])
plt.show()
